In [1]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 라이브러리 가져오기
import requests
import pandas as pd
import os
import dotenv  #환경변수 요청

In [3]:
#카카오 개발자 센터 접속 : http://developers.kakao.com/
#앱 > 앱생성 >생성된 앱 클릭 > 왼쪽메뉴 앱 선택 > 일반 선택 > 앱키에서 REST API 키 복사 ==> .env 넣어주기
#... > 왼쪽메뉴 앱 > 제품설정 > 카카오맵 : 사용설정에서 ON으로 선택 
# 문서>로컬> 이해하기 - 제공API
# 문서>로컬> REST API - header 확인

dotenv.load_dotenv()
my_key = os.getenv('KAKAO_RESTFULL_KEY')

In [ ]:
# 사이트별로 header 정보가 있기도 하고 없기도함.
# Kakao Map API 통해 위도, 경도 데이터 가져오는 함수를 정의


def get_geocoding(place):
    try:
        url = f"https://dapi.kakao.com/v2/local/search/address"
        params = {
            'query' : place
        }
        headers = {"Authorization": f"KakaoAK {my_key}"}
        response = requests.get(url, headers=headers, params=params)  #headers 없는 경우가 많음. 해당 URL에서 header 요구하면 넣기.
        result = response.json()
        return result["documents"][0]["y"], result["documents"][0]["x"]
    except:
        return pd.notna   #pd.notna : 결측치(데이터없음 또는 유효하지않는 데이터)

In [5]:
lat = []  #위도
lng = []  #경도

# 장소(주소) 리스트
places = ["서울특별시 종로구 세종대로 175", 
          "서울특별시 서초구 서초동 700", 
          "부산광역시 해운대구 해운대해변로 264"]

i=0
for place in places:   
    i = i + 1
    try:
        print(i, place)
        # get_geocoding 함수의 리턴값 호출하여 geo_location 변수에 저장
        place_lat, place_lon = get_geocoding(place)
        lat.append(place_lat)
        lng.append(place_lon)
        
    except:
        lat.append('')
        lng.append('')

# 데이터프레임으로 변환하기
df = pd.DataFrame({'위도':lat, '경도':lng}, index=places)

df

1 서울특별시 종로구 세종대로 175
2 서울특별시 서초구 서초동 700
3 부산광역시 해운대구 해운대해변로 264


,위도,경도
서울특별시 종로구 세종대로 175,37.5718478584908,126.976168275947
서울특별시 서초구 서초동 700,37.4810862955299,127.015245160054
부산광역시 해운대구 해운대해변로 264,35.1591069824231,129.160283786856


In [ ]:

#### 다른데이터를 가지고와서 주소에 위경도 정보 넣어보기 !!!!!!!!!!!!!!!!!!!!!!

df=pd.read_csv('https://raw.githubusercontent.com/pia222sk20/python/main/data/sample.csv', encoding='cp949')
df.head()
# df_simple = df[['상호명', '도로명주소']]
# df_simple.head()

df_simple = df.loc[:, ['상호명', '도로명주소']]
df_simple.head()

,상호명,도로명주소
0,하나산부인과,경기도 안산시 단원구 달미로 10
1,타워광명내과의원,서울특별시 강남구 언주로30길 39
2,조정현신경외과의원,경기도 시흥시 중심상가로 178
3,한귀원정신과의원,부산광역시 수영구 수영로 688
4,더블유스토어수지점,경기도 용인시 수지구 문정로 32


In [19]:
# 위도 경도 추가
# df_simple['위경도']=df_simple['도로명주소'].apply(lambda x: get_geocoding(x))
df_simple['도로명주소'].index

RangeIndex(start=0, stop=91335, step=1)

In [ ]:
import random
random_index = random.sample(range(91335), 50)
random_index

In [31]:
df_simple = df_simple.dropna()  #데이터 중에서 none 또는 NaN 없는 데이터 row를 삭제

In [11]:
df_simple_50 = df_simple.iloc[random_index]
results = [get_geocoding(addr) for addr in df_simple_50['도로명주소'].values]
results [:10]

[('37.346721724282', '126.732054712782'),
 ('34.7253990069815', '126.568008908384'),
 ('34.9483071964703', '127.521086536948'),
 ('36.3126899557356', '127.380308327292'),
 ('35.573927541891', '126.845512256934'),
 ('37.5051960660157', '127.025700266315'),
 ('35.7261060655733', '126.732004368793'),
 ('36.2989299146086', '127.319923688063'),
 ('35.8546940785106', '128.653349624526'),
 ('37.5886845716947', '127.236716768527')]

In [ ]:
df_simple_50.loc[:, '위도'], df_simple_50.loc[:, '경도'] = zip(*results)
 #zip(*) : 상기 (위도, 경도)를  위도 [] 경도[] 끼리 unpack.

In [18]:
df_simple_50.head()

,상호명,도로명주소,위도,경도
24825,서해동물병원,경기도 시흥시 중심상가로 146,37.346721724282,126.732054712782
22591,독일약국,전라남도 영암군 학산면 독천로 171-1,34.7253990069815,126.568008908384
10984,제일신통의원,전라남도 순천시 연향2로 55,34.9483071964703,127.521086536948
32082,김영유소아과의원,대전광역시 서구 도산로 45,36.3126899557356,127.380308327292
75901,우리들정형외과의원,전라북도 정읍시 중앙로 23,35.573927541891,126.845512256934


In [ ]:
# 상기 데이터를 지도에 표시하기 
%pip install folium

# https://python-visualization.github.io/folium/latest/

In [ ]:
import folium

m = folium.Map(location=(35.573927541891, 126.845512256934))  #최초지도 로드할 때, 중심좌표

folium.Marker(
    location=[35.573927541891, 126.845512256934],
    popup='우리들정형외과의원',
    tooltip='병원'
).add_to(m)

folium.Marker(
    location=[36.3126899557356, 127.380308327292],
    popup='김영유소아과의원',
    tooltip='병원'
).add_to(m)


m.save('map.html')   #html로 저장

In [28]:
for i in df_simple_50:
    print(i)    

상호명
도로명주소
위도
경도


In [29]:
for idx, row in df_simple_50.iterrows():
    print(idx, row['위도'], row['경도'])
    break

24825 37.346721724282 126.732054712782


In [30]:
import folium

m_all = folium.Map(location=(35.573927541891, 126.845512256934))  #최초지도 로드할 때, 중심좌표


for idx, row in df_simple_50.iterrows():   #iterrows 진짜 row를 가져옴
    # print (idx, row['위도'], row['경도'])
    
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['상호명'],
        tooltip=row['상호명'][-2:]
    ).add_to(m_all)

m_all.save('m_all.html')